In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math
import os
import sys
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
#from ml_metrics import rmsle
from sklearn.metrics import make_scorer
from xgboost import XGBRegressor

In [5]:
df_test = pd.read_csv('/kaggle/input/grupo-bimbo-inventory-demand/test.csv.zip')
print("Length test = ",len(df_test))
df_test.nunique()

Для простоты будем рассматривать только те данные, которые встречаются в test

In [6]:
#рассмотрим из каких данных состоит test.csv и сократим необходимый объем памяти
df_test.describe()

In [7]:
del[df_test]
dtype_test = {'Semana': np.uint8, 
         'Agencia_ID': np.uint16, 
         'Canal_ID': np.uint8,
         'Ruta_SAK': np.uint16, 
         'Cliente_ID': np.uint32, 
         'Producto_ID': np.uint16}
df_test=pd.read_csv('/kaggle/input/grupo-bimbo-inventory-demand/test.csv.zip', usecols=dtype_test.keys(), dtype=dtype_test)
df_test.head()

In [8]:
#посмотрим на целевую переменную
df_target = pd.read_csv('/kaggle/input/grupo-bimbo-inventory-demand/train.csv.zip', usecols={'Demanda_uni_equil'})
df_target.describe()

In [9]:
del[df_target]
dtype_train = {'Semana': np.uint8, 
         'Agencia_ID': np.uint16, 
         'Canal_ID': np.uint8,
         'Ruta_SAK': np.uint16, 
         'Cliente_ID': np.uint32, 
         'Producto_ID': np.uint16,
        'Demanda_uni_equil': np.uint16}
%time df_train = pd.read_csv('/kaggle/input/grupo-bimbo-inventory-demand/train.csv.zip', usecols=dtype_train.keys(), dtype=dtype_train)
df_train.head()

In [10]:
#переименуем для удобства
df_train = df_train.rename(columns = {'Semana':'Week_nbr', 'Agencia_ID':'Mall_ID','Canal_ID':'Sales_chanal_ID','Ruta_SAK':'Rout_ID','Cliente_ID':'Client_ID','Producto_ID':'Product_ID','Demanda_uni_equil':'Adjusted_Demand'},inplace = False)
df_train.head()

Рассмотрим распределение общего спроса по неделям

In [11]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

figure(figsize=(16, 6), dpi=80)
plt.yscale('log')
plt.title('Demand per week')

grouped_by_week = df_train[['Week_nbr', 'Adjusted_Demand']].groupby('Week_nbr').sum()
grouped_by_week.sort_values(by='Week_nbr')['Adjusted_Demand'].plot.bar()
plt.show()
del grouped_by_week

In [12]:
df_train.groupby('Week_nbr')['Adjusted_Demand'].count()

Рассмотрим распределение спроса для продуктов по неделям

In [13]:
len(df_train.groupby(['Product_ID','Week_nbr']).agg({'Adjusted_Demand':'sum'}))
Demand_for_prod_per_week=df_train.groupby(['Product_ID','Week_nbr'],as_index=False).agg({'Adjusted_Demand':'sum'})
Demand_for_prod_per_week.head(50)

In [ ]:
nbr_week_for_prod=df_train.groupby(['Product_ID'],as_index=False).agg({'Week_nbr':'nunique'})

In [ ]:
#показывает распределение продуктов по количеству недель, в которых они встречаются
nbr_week_for_prod.groupby('Week_nbr').agg({'Product_ID':'nunique'})

In [ ]:
#средний спрос по неделям для каждого продукта
mean_demand = Demand_for_prod_per_week.groupby('Product_ID',as_index=False).agg({'Adjusted_Demand':'mean'})

In [ ]:
figure(figsize=(16, 6), dpi=80)
plt.yscale('log')
plt.title('Log-scaled mean amount of product')

mean_demand.sort_values(by='Adjusted_Demand')['Adjusted_Demand'].plot.bar()
plt.show()

figure(figsize=(16, 6), dpi=80)
plt.title('Linear mean amount of product')

mean_demand.sort_values(by='Adjusted_Demand')['Adjusted_Demand'].plot.bar()
plt.show()

Рассмотрим распределение спроса по каналам

In [ ]:
Demand_on_chanal=df_train.groupby(['Sales_chanal_ID','Week_nbr'],as_index=False).agg({'Adjusted_Demand':'sum'})
Demand_on_chanal

In [ ]:
mean_demand_on_chanel=Demand_on_chanal.groupby('Sales_chanal_ID',as_index=False).agg({'Adjusted_Demand':'mean'})
figure(figsize=(16, 6), dpi=80)
plt.yscale('log')
plt.title('Log-scaled mean amount of demand on chanel')

mean_demand_on_chanel.sort_values(by='Adjusted_Demand')['Adjusted_Demand'].plot.bar()
plt.show()

figure(figsize=(16, 6), dpi=80)
plt.title('Linear mean amount of demand on chanel')

mean_demand_on_chanel.sort_values(by='Adjusted_Demand')['Adjusted_Demand'].plot.bar()
plt.show()

Присоединим дополнительные таблицы и посмотрим, что можно посмотреть с их помощью. Рассмотрим распределение по городам и штатам

In [ ]:
df_town = pd.read_csv('../input/grupo-bimbo-inventory-demand/town_state.csv.zip')
df_town = df_town.rename(columns={'Agencia_ID':'Mall_ID'})
df_town.head()

In [ ]:
print("Количество ТЦ: ",len(df_town))
df_town.nunique()

In [ ]:
len(df_town.groupby(["Town","State"]))

In [ ]:
df_train = df_train.merge(df_town, how='inner', on='Mall_ID')

In [ ]:
df = df_train.copy()
fig = plt.figure(figsize=(8,8))
df.State.value_counts().plot(kind='barh')
plt.title("Hist of State")
plt.grid(True)
plt.show()

In [ ]:
cnt = df.State.value_counts()
big_category = set(cnt[cnt >= 2100].index)

df['State'] = df.State.apply(lambda w: w if w in big_category else 'another')
fig = plt.figure(figsize=(8,5))
df.State.value_counts().plot(kind='barh')
plt.title("Hist of State")
plt.show() 

In [ ]:
fig = plt.figure(figsize=(8,8))
df.Town.value_counts().plot(kind='barh')
plt.title("Hist of Town")
plt.show()

Рассмотрим также продуктовую таблицу

In [ ]:
df_product = pd.read_csv('../input/grupo-bimbo-inventory-demand/producto_tabla.csv.zip')
df_town = df_town.rename(columns={'Agencia_ID':'Mall_ID'})
df_town.head()